# Série 1

Ceci est une série d'exercices sur les calculs homomorphiques en utilisant des courbes elliptiques.

# Exercice 1

On va commencer par deux classes qui vont nous permettre de plonger un peu dans le merveilleux
monde des courbes elliptiques:

- `Scalar` - qui représente un scalaire. Il est souvent associé à la clé privée
- `Point` - qui représente un point sur la courbe. Le plus souvent associé à la clé publique

Ces deux classes vont nous permettre de faire les opérations nécessaires pour faire de la cryptographie
homomorphique.

## 1. Connaissance

On va simplement faire quelques calculs avec les points. 
En interne la classe fait les pas nécessaires pour passer d'un nombre entier à un scalaire, puis pour créer un point.

Suivez les points suivants dans le fichier ecc.py et notez les methodes utilisées:
- set_int de la classe Point
- get_int de la classe Point

Vous pouvez aussi essayer quelques exemples vous-mêmes.

** Questions **

1. Que se passe-t-il si on additionne 250 + 250 et qu'on veut montrer le résultat avec get_int? Pourquoi? 
2. Comment pouvez-vous changer ceci?

In [ ]:
from ecc import Point, Scalar

a = Point().set_int(10)
for i in range(21):
    b = Point().set_int(i)
    print("10 -", i, "is", a.sub(b).get_int())
    
c = Point().set_int(250)
print(c.add(c).get_int())

### Réponses

1. Si on additionne 250 et 250, on reçoit un `ValueError`. Ceci est dû au fait que la
  méthode `get_int` ne cherche que les valeurs -499 à 499.
2. Pour palier à cette limite, on peut changer la valeur dans `ecc.py`, dans la classe `Point`,   dans la méthode `get_int`.
  Attention: pour activer les changements, il faut recharger le *Kernel* de Jupyter.

## 2. Compréhension

Maintenant que nous avons les classes Point et Scalar, on peut les utiliser pour implémenter le chiffrement et
déchiffrement El Gamal.

Dans le code suivant, la class `ElGamal` peut être initialisé avec la méthode statique `ElGamal.encrypt`.
Une fois qu'on a un objet ElGamal, on peut appeler les méthodes `decrypt` pour retrouver le message original,
et `add` pour ajouter deux objets ElGamal d'une façon homomorphique.

Dans l'exemple ici, on fait
1. Créer une pair de clés privé/publique
2. Créer un message "123" et le chiffrer
3. Déchiffrer le message et afficher le contenu

A vous de jouer:
- créer un deuxième message "234" et le chiffrer avec la même clé publique que le premier
- additionner les deux messages
- les dechiffrer
- afficher le résultat

Essayez de faire un troisième message "345" et l'ajoutez aux deux premiers.
- Qu'est-ce que vous observez? Pourquoi?

In [ ]:
# Exercice 1 - Partie 2
from __future__ import annotations

class ElGamal:
    def __init__(self, C1: Point, C2: Point):
        self.C1 = C1
        self.C2 = C2

    def decrypt(self, priv: Scalar) -> Point:
        S = self.C1.scalarmult(priv)
        M = self.C2.sub(S)
        return M
    
    def add(self, other: ElGamal) -> ElGamal:
        return ElGamal(self.C1.add(other.C1), self.C2.add(other.C2))

    @staticmethod
    def encrypt(pub: Point, msg: Point) -> ElGamal:
        y = Scalar().rnd()
        C1 = Point.scalarmult_base(y)
        S = pub.scalarmult(y)
        C2 = msg.add(S)
        return ElGamal(C1, C2)
    
# Create random private key
priv = Scalar().rnd()
# Calculate the corresponding public key by multiplying with the base point
pub = Point.scalarmult_base(priv)

# Encrypt the message "123"
msg1 = Point().set_int(123)
eg = ElGamal.encrypt(pub, msg1)

# Decrypt the message and read the point
msg1p = eg.decrypt(priv)
print("Message is:", msg1p.get_int())

# Encrypt a second message "234"
msg2 = Point().set_int(234)
eg2 = ElGamal.encrypt(pub, msg2)
eg12 = eg.add(eg2)

# Decrypt the message and read the point
print("Result of message 1 + 2 is:", eg12.decrypt(priv).get_int())

# Encrypt a third message "345"
eg3 = ElGamal.encrypt(pub, Point().set_int(234))
eg123 = eg.add(eg2).add(eg3)

# Decrypt the message and read the point
print("Result of messages 1 + 2 + 3 is:", eg123.decrypt(priv).get_int())

### Réponse

On rencontre le même problème qu'avant: le nombre final est trop élevé, et donc la méthode `get_int` ne peut pas le décoder.

## 3. Application

Finalement on peut s'attaquer à un système décentralisé. On va faire simple:

- deux nœuds avec chacun sa clé privée / publique
- deux messages, 77 et -34, qui seront chiffrés et additionnés

Dans le code en bas il y a les commentaires pour les différentes étapes à faire.

In [ ]:
# Exercice 1 - Partie 3

# créer deux pairs de clés privées / publique (priv1, priv2, pub1, pub2)

# First hospital
priv1 = Scalar().rnd()
pub1 = Point.scalarmult_base(priv1)

# Second hospital
priv2 = Scalar().rnd()
pub2 = Point.scalarmult_base(priv2)

# calculer la clé publique agrégée (pub), sans passer par les clés privées!
pub_agg = pub1.add(pub2)

#   vous pouvez utiliser les clés privées pour vérifier que c'est juste, 
#   mais dans un vrai système on ne va seulement utiliser les clés publiques
pub_agg_from_private = Point.scalarmult_base(priv1.add(priv2))
print("Are the public keys the same?", pub_agg.hex() == pub_agg_from_private.hex())

# chiffrer deux messages (77, -34) avec la clé publique agrégée pour obtenir 
msg1 = Point().set_int(77)
eg1 = ElGamal.encrypt(pub_agg, msg1)

msg2 = Point().set_int(-35)
eg2 = ElGamal.encrypt(pub_agg, msg2)

# additionner les deux messages
eg_tot = eg1.add(eg2)

# calculer la participation au déchiffrement de la clé priv1
d1 = eg_tot.C1.scalarmult(priv1)

# calculer la participation au déchiffrement de la clé priv2
d2 = eg_tot.C1.scalarmult(priv2)

# calculer manuellement le déchiffrement finale en utilisant les deux participations
#   alternativement vous pouvez ajouter une nouvelle méthode à ElGamal qui fait le déchiffrement
#   partielle, C_2 = C_2 - C_1 · x_i
#   après l'application de la clé privée 1 et la clé privée 2, C_2 contient le message
d_tot = d1.add(d2)
point_tot = eg_tot.C2.sub(d_tot)

# afficher le résultat
print("The final result is:", point_tot.get_int())